In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import requests
from datetime import datetime as dt

In [2]:
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
assert ALPHAVANTAGE_KEY is not None, "You must export ALPHAVANTAGE_KEY..."
ALPHAVANTAGE_KEY

'032HZXCB7T21N7SC'

In [3]:
SAMPLE_START_DATE = dt.fromisoformat('2019-12-19').date()
SAMPLE_END_DATE = dt.fromisoformat('2023-11-30').date()
(SAMPLE_START_DATE, SAMPLE_END_DATE)

(datetime.date(2019, 12, 19), datetime.date(2023, 11, 30))

In [4]:
DIGITAL_CURRENCY_URL = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={}&datatype=csv'.format(ALPHAVANTAGE_KEY)
SPY_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='SPY', apikey=ALPHAVANTAGE_KEY)
QQQ_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='QQQ', apikey=ALPHAVANTAGE_KEY)
(DIGITAL_CURRENCY_URL, SPY_URL, QQQ_URL)

('https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=QQQ&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv')

In [5]:
%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format
sns.set_style("whitegrid")

mpl.rcParams.update({
                     'text.color' : "white",
                     'lines.linewidth': 3,
                     'font.size': 16.0,
                     'legend.facecolor': "182742",
                     'legend.edgecolor': 'CFB023',
                     'legend.fancybox': True,
                     'legend.framealpha': 1.,
                     'legend.loc': 'right',
                     'xtick.color': 'white',
                     'ytick.color': 'white',
                     'ytick.minor.visible': True,
                     'axes.facecolor': '#182742',
                     'axes.edgecolor': 'white',
#                      'axes.grid.axis': 'y',
#                      'axes.grid.which': 'minor',
                     'axes.labelcolor' : "white",
                     'axes.labelsize': "24",
                     'axes.labelweight': "bold",
                     'axes.titlesize': 28,
                     'axes.titleweight': 'bold',
                     'axes.titlepad': 20,
                     'figure.facecolor': '#182742',
#                      'xtick.bottom': False,
                    }
                   )

In [6]:
def total_return(prices):
  return prices.iloc[-1] / prices.iloc[0] - 1

# Download Bitcoin Historical Data

In [7]:
# TODO: Figure out how to add error checking
# r = requests.get(URL)
# data = r.json()
# assert 'Error Message' not in data.keys(), "Error Received. {}".format(data['Error Message'])

df = pd.read_csv(DIGITAL_CURRENCY_URL)
df['timestamp'] = pd.to_datetime(df.timestamp)
df.index = df.timestamp.dt.date
df = df[::-1]
df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
df['pct_change'] = df['btc_close'].pct_change()
df[['timestamp', 'btc_close', 'pct_change']].to_csv('bitcoin_daily_returns.csv')
df[['timestamp', 'btc_close', 'pct_change']]
df

,timestamp,btc_close,volume,market cap (USD),pct_change
timestamp,,,,,
2021-03-12,2021-03-12,"57,221.7200","73,405.4060","73,405.4060",NaN
2021-03-13,2021-03-13,"61,188.3900","83,245.0913","83,245.0913",0.0693
2021-03-14,2021-03-14,"58,968.3100","52,601.0528","52,601.0528",-0.0363
2021-03-15,2021-03-15,"55,605.2000","102,771.4273","102,771.4273",-0.0570
2021-03-16,2021-03-16,"56,900.7500","77,986.6944","77,986.6944",0.0233
...,...,...,...,...,...
2023-12-02,2023-12-02,"39,450.3500","26,696.9216","26,696.9216",0.0198
2023-12-03,2023-12-03,"39,972.2600","26,710.6534","26,710.6534",0.0132
2023-12-04,2023-12-04,"41,991.1000","79,272.3306","79,272.3306",0.0505


In [8]:
btc_monthly_returns = df.groupby([df.timestamp.dt.year, df.timestamp.dt.month])['btc_close'].apply(total_return)
btc_monthly_returns.to_csv("bitcoin_monthly_returns.csv")
btc_monthly_returns

timestamp  timestamp
2021       3            0.0265
           4           -0.0175
           5           -0.3555
           6           -0.0449
           7            0.2375
           8            0.1821
           9           -0.1022
           10           0.2733
           11          -0.0650
           12          -0.1918
2022       1           -0.1939
           2            0.1154
           3            0.0245
           4           -0.1869
           5           -0.1733
           6           -0.3309
           7            0.2082
           8           -0.1383
           9           -0.0352
           10           0.0611
           11          -0.1621
           12          -0.0256
2023       1            0.3917
           2           -0.0249
           3            0.2047
           4            0.0274
           5           -0.0306
           6            0.1363
           7           -0.0443
           8           -0.1267
           9            0.0449
           10     

In [9]:

def read_equities(url, start_date, end_date, outfile):
    df = pd.read_csv(url)
    df['timestamp'] = pd.to_datetime(df.timestamp)
    df.index = df.timestamp.dt.date
    df = df[::-1]
    sample = df[start_date:end_date]
    # df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
    # df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
    sample['pct_change'] = sample['close'].pct_change().copy()
    sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
    sample['growth_of_10k'].iloc[0] = 10000.
#     sample.insert(4, 'growth_of_10k', 0.)
#     sample['growth_of_10k'].iloc[0] = 10000.0
# #     print(sample.iloc[1:]['growth_of_10k'])
#     print(sample['growth_of_10k'].shift(1))
#     print((1 + sample['pct_change']))
#     sample['growth_of_10k'] = sample['growth_of_10k'].shift(1) * (1 + sample['pct_change'])
    sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']].to_csv(outfile)
    return sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']]

In [10]:
spy_history = read_equities(SPY_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'spy_history.csv')
spy_history

/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,320.9000,NaN,"10,000.0000"
2019-12-20,2019-12-20,320.7300,-0.0005,"9,994.7024"
2019-12-23,2019-12-23,321.2200,0.0015,"10,009.9720"
2019-12-24,2019-12-24,321.2300,0.0000,"10,010.2836"
2019-12-26,2019-12-26,322.9400,0.0053,"10,063.5712"
...,...,...,...,...
2023-11-24,2023-11-24,455.3000,0.0006,"14,188.2206"
2023-11-27,2023-11-27,454.4800,-0.0018,"14,162.6675"
2023-11-28,2023-11-28,454.9300,0.0010,"14,176.6906"


In [11]:
qqq_history = read_equities(QQQ_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'qqq_history.csv')
qqq_history

/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_60367/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,210.8600,NaN,"10,000.0000"
2019-12-20,2019-12-20,211.7100,0.0040,"10,040.3111"
2019-12-23,2019-12-23,211.8100,0.0005,"10,045.0536"
2019-12-24,2019-12-24,211.9200,0.0005,"10,050.2703"
2019-12-26,2019-12-26,213.7900,0.0088,"10,138.9548"
...,...,...,...,...
2023-11-24,2023-11-24,389.5100,-0.0014,"18,472.4462"
2023-11-27,2023-11-27,389.1700,-0.0009,"18,456.3217"
2023-11-28,2023-11-28,390.1900,0.0026,"18,504.6951"
